# Before your start:
- Read the README.md file
- Comment as much as you can and use the resources in the README.md file
- Happy learning!

# Challenge 1 - Accessing our database
Create a connection to access the sakila database with the ip, user and password provided in class. Take a look at the data. Do some joins in order to have more elaborated tables in a dataframe (you can try first the query in the DBMS and then use it in your notebook).


In [14]:
# your code here
# I am importing database from Octavio's slack link. Database is saved on 'data' folder

from sqlalchemy import create_engine

engine = create_engine('sqlite:///../data/juliarochflores.db')

query = """
SELECT business_info.id, personal_info.lastName, rank_info.name, rank_info.position FROM business_info 
LEFT JOIN personal_info ON personal_info.id = business_info.id
LEFT JOIN rank_info ON rank_info.id = business_info.id;
"""

pd.read_sql_query(query, engine)



,id,lastName,name,position
0,1001,plATt,michaEL Platt,241.0
1,1005,KOO,koo BON-SIK,2159.0
2,1008,toELLer,torsteN TOEller,1468.0
3,1015,snYDER,daN SNYDER,1098.0
4,1017,kohLER,"herbert kOHLER, JR.",165.0
...,...,...,...,...
2203,9981,feRTITTa,loRENZO FERtitta,1116.0
2204,9983,shaMALOv,kirill SHAMALov,1726.0
2205,9988,aLEXANDEr,leslie aLEXANDER,1157.0
2206,9996,al ghURAIr,abdulla bin AHMAD AL GHURAIr,296.0


# Challenge 2 - Working with JSON files

Import the pandas library.

In [16]:
# your code here
import pandas as pd

In the next cell, load the data in `nasa.json` in the `data` folder and load it into a pandas dataframe. Name the dataframe `nasa`.

In [21]:
# your code here

nasa = pd.read_json('../data/nasa.json')

Now that we have loaded the data, let's examine it using the `head()` function.

In [22]:
# your code here

nasa.head()

,:@computed_region_cbhk_fwbd,:@computed_region_nnqa_25f4,fall,geolocation,id,mass,name,nametype,recclass,reclat,reclong,year
0,NaN,NaN,Fell,"{'type': 'Point', 'coordinates': [6.08333, 50....",1,21.0,Aachen,Valid,L5,50.77500,6.08333,1880-01-01T00:00:00.000
1,NaN,NaN,Fell,"{'type': 'Point', 'coordinates': [10.23333, 56...",2,720.0,Aarhus,Valid,H6,56.18333,10.23333,1951-01-01T00:00:00.000
2,NaN,NaN,Fell,"{'type': 'Point', 'coordinates': [-113, 54.216...",6,107000.0,Abee,Valid,EH4,54.21667,-113.00000,1952-01-01T00:00:00.000
3,NaN,NaN,Fell,"{'type': 'Point', 'coordinates': [-99.9, 16.88...",10,1914.0,Acapulco,Valid,Acapulcoite,16.88333,-99.90000,1976-01-01T00:00:00.000
4,NaN,NaN,Fell,"{'type': 'Point', 'coordinates': [-64.95, -33....",370,780.0,Achiras,Valid,L6,-33.16667,-64.95000,1902-01-01T00:00:00.000


#### The `value_counts()` function is commonly used in pandas to find the frequency of every value in a column.

In the cell below, use the `value_counts()` function to determine the frequency of all types of asteroid landings by applying the function to the `fall` column.

In [23]:
# your code here

nasa['fall'].value_counts()

Fell     996
Found      4
Name: fall, dtype: int64

Finally, let's save the dataframe as a json file again. Save the dataframe using the `orient=records` argument and name the file `nasa-output.json`. Remember to save the file inside the `data` folder.

In [26]:
# your code here

nasa.to_json(f'../data/nasa-output.json', orient='records')

# Challenge 2 - Working with CSV and Other Separated Files

CSV files are more commonly used as dataframes. In the cell below, load the file from the URL provided using the `read_csv()` function in pandas. Starting version 0.19 of pandas, you can load a CSV file into a dataframe directly from a URL without having to load the file first and then transform it. The dataset we will be using contains information about NASA shuttles.

In the cell below, we define the column names and the URL of the data. Following this cell, read the tst file to a variable called `shuttle`. Since the file does not contain the column names, you must add them yourself using the column names declared in `cols` using the `names` argument. Additionally, a tst file is space separated, make sure you pass ` sep=' '` to the function.

In [67]:
cols = ['time', 'rad_flow', 'fpv_close', 'fpv_open', 'high', 'bypass', 'bpv_close', 'bpv_open', 'class', 'col10']
tst_url = 'https://archive.ics.uci.edu/ml/machine-learning-databases/statlog/shuttle/shuttle.tst'

In [68]:
# your code here
shuttle = pd.read_csv(tst_url, sep=' ')
shuttle.columns = cols

Let's verify that this worked by looking at the `head()` function.

In [69]:
# your code here
shuttle.head()

,time,rad_flow,fpv_close,fpv_open,high,bypass,bpv_close,bpv_open,class,col10
0,56,0,96,0,52,-4,40,44,4,4
1,50,-1,89,-7,50,0,39,40,2,1
2,53,9,79,0,42,-2,25,37,12,4
3,55,2,82,0,54,-6,26,28,2,1
4,41,0,84,3,38,-4,43,45,2,1


To make life easier for us, let's turn this dataframe into a comma separated file by saving it using the `to_csv()` function. Save `shuttle` into the file `shuttle.csv` and ensure the file is comma separated, that we are not saving the index column and that the file is saved inside the `data` folder.

In [74]:
# your code here

shuttle.to_csv('../data/shuttle.csv', sep= ',',index=False)

# Challenge 3 - Working with Excel Files

We can also use pandas to convert excel spreadsheets to dataframes. Let's use the `read_excel()` function. In this case, `astronauts.xls` is in the `data` folder. Read this file into a variable called `astronaut`. 

Note: Make sure to install the `xlrd` library if it is not yet installed.

In [11]:
# your code here

Use the `head()` function to inspect the dataframe.

In [12]:
# your code here

Use the `value_counts()` function to find the most popular undergraduate major among all astronauts.

In [13]:
# your code here

Due to all the commas present in the cells of this file, let's save it as a tab separated csv file. In the cell below, save `astronaut` as a **tab separated file** using the `to_csv` function. Call the file `astronaut.csv`. Remember to remove the index column and save the file in the `data` folder.

In [14]:
# your code here

# Bonus Challenge - Fertility Dataset

Visit the following [URL](https://archive.ics.uci.edu/ml/datasets/Fertility) and retrieve the dataset as well as the column headers. Determine the correct separator and read the file into a variable called `fertility`. Examine the dataframe using the `head()` function. 

Look in Google for a way to retrieve this data!

In [15]:
# your code here